## Fixing Broken Data

#### Overwriting the passing, rushing, receiving base tables because NFL week is incorrect

In [22]:
# importing various libraries
import gspread
# Service client credential from oauth2client
from oauth2client.service_account import ServiceAccountCredentials
# Create scope
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
# create some credential using that scope and content of startup_funding.json
creds = ServiceAccountCredentials.from_json_keyfile_name(
    '../quickstart/g_sheet_creds.json', scope)
# create gspread authorize using that credential
client = gspread.authorize(creds)
my_email = 'matthewjchristy66@gmail.com'


In [23]:
# data manipulation
import pandas as pd
import numpy as np
from requests import get
import lxml.html as lh


In [25]:
base_url = 'http://www.nfl.com/stats/weeklyleaders'

weeks = [x for x in range(1, 18)]

stat_cat = ['Passing', 'Rushing', 'Receiving']

urls = []

names = []

for i in weeks:
    for j in stat_cat:
        full_url = base_url + '?week=' + \
            str(i) + '&season=2019' + '&showCategory=' + j
        urls.append(full_url)
        name = '2019_' + 'week_' + str(i) + '_' + j
        names.append(name)


In [ ]:
from data_acq_help_functions import *


In [ ]:
def over_writer(data, sheet_name):
    # Grabbing Parameters for looping
    n_rows = data.shape[0]
    n_cols = data.shape[1]

    # getting cell list to batch update
    import string
    end_col = string.ascii_uppercase[n_cols - 1]
    end_row = n_rows + 1

    sheet_range = 'A1:' + end_col + str(end_row)

    # turning df to one long list
    df_as_list = data.stack().tolist()
    df_as_list = data.columns.tolist() + df_as_list

    # getting the target sheet
    sheet = client.open(sheet_name)
    ws = sheet.get_worksheet(0)
    cell_list = ws.range(sheet_range)

    # writing df list to cell range list
    for i in range(0, len(cell_list)):
        cell_list[i].value = df_as_list[i]

    # batch updating
    ws.update_cells(cell_list)


In [32]:
# making empty dataframes
passing = pd.DataFrame()
receiving = pd.DataFrame()
rushing = pd.DataFrame()


In [33]:
for i in range(0, len(urls)):
    source_html = get_url(urls[i])
    colnames = get_headers(source_html)
    table_data = get_table_data(source_html, colnames)
    raw_df = create_df(table_data)
    df = clean_df(raw_df, urls[i])

    if 'Passing' in urls[i]:
        passing = pd.concat([passing, df])

    elif 'Receiving' in urls[i]:
        receiving = pd.concat([receiving, df])

    else:
        rushing = pd.concat([rushing, df])


### fixing the data 

In [34]:
over_writer(data=passing, sheet_name='passing')


In [35]:
over_writer(data=rushing, sheet_name='rushing')


In [36]:
over_writer(data=receiving, sheet_name='receiving')
